In [1]:
import tensorflow as tf
import numpy as np

import warnings
## Tensorflow produces a lot of warnings. We generally want to suppress them. The below code does exactly that. 
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

## Using predefined estimators
In this notebook we recreate the neural network defined first in notebook [3.Neural-Networks.ipynb](3.Neural-Networks.ipynb)  
using the predefined estimator `DNNClassifier`

## Benefits of Estimators

- Estimator-based models are independent of operating environment
    - local host
    - GPUs
    - CPU clusters

### More benefits
- Simplify model sharing between developers
- State of the art model architectures with more intuitive high-level code

Consult https://www.tensorflow.org/programmers_guide/estimators for more advantages of using Estimators as described by the developers of TensorFlow.

## Read Data
The MNist dataset is available from within TensorFlow tutorials.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

# Wrap input as a function (THE "input function" will be defined below)
def input(dataset):
    return dataset.images, dataset.labels.astype(np.int32)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


##  Define feature columns

In [3]:
# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]

## Define Neural Network

In [4]:
# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 256],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir="./tmp/mnist_model_256_256"   # Location for storing checkpoints.
)

## Define training input function
* Supplies data for training, evaluation, prediction
* Should yield tuples of:
    - Python dict `features`: key = name of feature, value = array of feature values
    - Array `label` : label for every example

In [5]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.train)[0]},
    y=input(mnist.train)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

## Train the neural network
* Checkpoint used for "warm start"
* Checkpoints saved

In [6]:
tf.logging.set_verbosity(tf.logging.INFO)
classifier.train(input_fn=train_input_fn, steps=10000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into ./tmp/mnist_model_256_256/model.ckpt.
INFO:tensorflow:loss = 122.352196, step = 1
INFO:tenso

In [7]:
# Have a look at the checkpoint directory.
!ls -lrt ./tmp/mnist_model_256_256/ | tail

-rw-r--r--  1 brianroepke  staff       16 Nov  2 08:08 model.ckpt-0.data-00000-of-00002
-rw-r--r--  1 brianroepke  staff  3231864 Nov  2 08:08 model.ckpt-0.data-00001-of-00002
-rw-r--r--  1 brianroepke  staff      839 Nov  2 08:08 model.ckpt-0.index
-rw-r--r--  1 brianroepke  staff   175517 Nov  2 08:08 model.ckpt-0.meta
-rw-r--r--  1 brianroepke  staff       16 Nov  2 08:08 model.ckpt-10000.data-00000-of-00002
-rw-r--r--  1 brianroepke  staff  3231864 Nov  2 08:08 model.ckpt-10000.data-00001-of-00002
-rw-r--r--  1 brianroepke  staff      839 Nov  2 08:08 model.ckpt-10000.index
-rw-r--r--  1 brianroepke  staff      132 Nov  2 08:08 checkpoint
-rw-r--r--  1 brianroepke  staff   175517 Nov  2 08:08 model.ckpt-10000.meta
-rw-r--r--  1 brianroepke  staff  1131813 Nov  2 08:08 events.out.tfevents.1572707320.Brians-iMac.local


## Define test input function

In [8]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.test)[0]},
    y=input(mnist.test)[1],
    num_epochs=1,
    shuffle=False
)

## Evaluate accuracy

In [9]:
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-02T08:08:59Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_model_256_256/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-02-08:08:59
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.9754, average_loss = 0.08156739, global_step = 10000, loss = 10.3249855
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: ./tmp/mnist_model_256_256/model.ckpt-10000

Test Accuracy: 97.539997%

